<a href="https://colab.research.google.com/github/DinhoVCO/Mineria-de-datos/blob/main/APRIORI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **REGLAS DE ASOCIACIÓN**
---
**UNIVERSIDAD NACIONAL DE SAN ANTONIO ABAD DEL CUSCO**


>**ESCUELA PROFESIONAL :** Ing. Informatica y de Sistemas



> **CURSO :**  Minería de datos💻


> **ALUMNO :** Vega Centeno Olivera, Ronaldinho🧑









##**1.-Instalar bibliotecas**

In [1]:
import numpy as np          #para procesar el dataset
import statistics as stats  #para hallar medidas estadisticas
from itertools import chain, combinations #para generar las combinarciones de los frequentItemSets
import time                 # para medir el tiempo de ejecucion

## **2.-Importar datos**
La base de datos a utilizar corresponde a múltiples playlists de la plataforma spotify creadas por usuarios de esta.

In [2]:
#Importamos el archivo .npy en la variable data de tipo numpy.ndarray 0-dimensional
data = np.load('DB/spotify.npy',allow_pickle=True)
#convertimos a una lista -> obtenemos un diccionario de 10000 playlists
data=data.tolist()


##**3.- Preprocesar (lista de listas)**

In [3]:
def dataset_preprocessing(data):
  """
  Genera una lista de canciones unicas y de playlist sin canciones repetidas.
  
  Parameters
  =========
    `data :` diccionario  
          ej -> {0:['A', 'B'],1:['B', 'C']} 

  Returns
  =========
    `playlist :` Lista de elementos del tipo frozenset() 
          ej -> [frozenset({'A'}),frozenset({'B'}),frozenset({'C'})]
          
    `listSongs :`Lista de conjuntos
          ej -> [{'A', 'B'},{'B', 'C'}]
          
  """
  playlist=[]
  songs=set()
  for i in data:
    playlist.append(set(data[i]))
    songs|=set(data[i])
  songs=[[i] for i in list(songs)]
  songs.sort()
  # Utilizamos frozenset() para que sea inmutable
  # y poder trabajar con diccionarios
  listSongs=list(map(frozenset,songs))
  return listSongs, playlist

In [4]:
canciones, listas = dataset_preprocessing(data)
print('Total de canciones distintas:', len(canciones))
print('Total de Playlists(transaccciones):', len(listas))

Total de canciones distintas: 132920
Total de Playlists(transaccciones): 10000


In [5]:
#Ejemplo de tres canciones
canciones[:3]

[frozenset({'!Dios Mio! El Diablo Es Muy Picante'}),
 frozenset({'"*!*!" What U Heard'}),
 frozenset({'"2014: A Pesach Medley"'})]

In [6]:
#Plailist n°999
listas[999]

{'A Quién Le Importa',
 'Amor De Pobre - No Tengo Dinero (Hip-Hop Version)',
 'El Hombre Es Como El Oso',
 'El Mario de Tu Mujer (feat. Don Miguelo)',
 'El Regeton',
 'El Tao Tao',
 'Juana La Cubana',
 'La Bomba',
 'La Mesa Que Mas Aplauda (Za Za Za)',
 'La Vaca',
 'Muévelo',
 'Party In Miami',
 'Que Lo Que',
 'Tu Pun Pun',
 'Un Ladrón'}

## **4.-Definir umbrales** 🟠⏮ 

In [8]:
"""
Obtenemos la frecuencia de cada cancion en el dataset
para poder obtener sus medidas de tendencia central
"""
frequencies=dict()
for i in listas:
  for j in i:
    frequencies[j] =frequencies.get(j, 0) + 1
frecuencias=list(frequencies.values())
frecuencias.sort()

In [9]:
print("MEDIDAS DE TENDENCIA CENTRAL Y DISPERSIÓN")
print("-----------------------------------------")
print("Media aritmética     : ",stats.mean(frecuencias))
print("Mediana              : ",stats.median(frecuencias)) 
print("Moda                 : ",stats.mode(frecuencias)) 
print("Varianza poblacional : ",stats.pvariance(frecuencias)) 
#La varianza mide en qué medida se dispersan los valores de una muestra alrededor de un valor central.

MEDIDAS DE TENDENCIA CENTRAL Y DISPERSIÓN
-----------------------------------------
Media aritmética     :  4.910028588624737
Mediana              :  1.0
Moda                 :  1
Varianza poblacional :  254.75020336963107


In [11]:
lon_min = 2 # Número mínimo de canciones a considerar en las reglas
veces_minimas_aparece_en_lista=100 #Support count
soporte_min = veces_minimas_aparece_en_lista / len(listas) #Support
confianza_min =0.5
sustento_min =1.5

## **5.-Algoritmo APRIORI** 

Para el desarrollo de los modulos se trabajo con estructuras como listas, conjuntos,conjuntos inmutables y diciconarios.

```
-list()
-set()
-frozenset()
-dict()
```
En algunos casos son listas de listas o listas de conjuntos.

Como la clave ( key ) de los dicicionarios deben ser inmutables se utilizó `frozenset()` que son conjuntos inmutables y asi poder trabajar los conjuntos como `key` y los supports como `values`, además que podemos realizar operaciones entre conjuntos, lo que facilita el desarrolo del código.




### **5.1. Get frequent items sets**

#### 5.1.1. get_first_frequent_itemset

In [12]:
# Modulo optimo para obtener 1-itemsetfrequents
def get_first_frequent_itemset(transactions, k_itemSet,minSupport):
  """
  Halla 1-itemsetfrequents y el support de cada conjunto.

  Parameters
  =========
    `transactions :` Lista de conjuntos
          ej -> [{'A', 'B'},{'B', 'C'}]

    `k_itemSet :` Lista de elementos del tipo frozenset() (1 elemento)
          ej -> [frozenset({'A'}),frozenset({'B'}),frozenset({'C'})]

    `minSupport :` Soporte minimo para las relaciones (float)
          ej -> 0.5
  Returns
  =========
    `ItemSetFrequent :` Lista de elementos del tipo frozenset() (1 elemento)
          ej -> [frozenset({'A'}),frozenset({'B'}),frozenset({'C'})]
          
    `supportItemSet :` Dicionario
          ej -> {'A':0.5 , 'B':0.6 , 'C':0.8 }
          
  """
  #Calculamos el supportcount (frecuencia) de cada subconjunto
  frequencies=dict()
  for i in transactions:
    for j in i:
      frequencies[j] =frequencies.get(j, 0) + 1
  #Calculamos el support y comprobamos que sea mayor a minSupport
  TotalTransactions = float(len(transactions))
  ItemSetFrequent = []
  supportItemSet = {}
  for key in frequencies:
      support = frequencies[key] / TotalTransactions  # Calcular soporte
      if support >= minSupport:  # Si el soporte es mayor que el soporte mínimo establecido
          ItemSetFrequent.insert(0,key)
      supportItemSet[key] = support
  ItemSetFrequent.sort()
  ItemSetFrequent=[frozenset({i}) for i in ItemSetFrequent ]
  return ItemSetFrequent, supportItemSet

#### 5.1.2. get_k_frequent_itemset

In [13]:
def get_k_frequent_itemset(transactions, k_itemSet,minSupport):
  """
  Halla k-itemsetfrequents y el support de cada conjunto.

  Parameters
  =========
    `transactions :` Lista de conjuntos
          ej -> [{'A', 'B'},{'B', 'C'}]

    `k_itemSet :` Lista de elementos del tipo frozenset() (k elementos)
          ej -> [frozenset({'A',...}),frozenset({'B',...}),frozenset({'C',...})]

    `minSupport :` Soporte minimo para las relaciones (float)
          ej -> 0.5
  Returns
  =========
    `ItemSetFrequent :` Lista de elementos del tipo frozenset() (k elementos)
          ej -> [frozenset({'A',...}),frozenset({'B',...})]
          
    `supportItemSet :` Dicionario de conjuntos inmutables frozenset() (k elemento)
          ej -> {frozenset({'A',...}): 0.5 , frozenset({'B',...}): 0.6}
          
  """
  #Calculamos el supportcount (frecuencia) de cada subconjunto
  frequencies=dict()
  for i in transactions:
    for j in k_itemSet:
      if (j<=i):
        frequencies[j] =frequencies.get(j, 0) + 1
  #Calculamos el support y comprobamos que sea mayor a minSupport
  TotalTransactions = float(len(transactions))
  ItemSetFrequent = []
  supportItemSet = {}
  for key in frequencies:
      support = frequencies[key] / TotalTransactions  # Calcular soporte
      if support >= minSupport:  # Si el soporte es mayor que el soporte mínimo establecido
          ItemSetFrequent.insert(0,key)
      supportItemSet[key] = support
  ItemSetFrequent.sort()
  return ItemSetFrequent, supportItemSet

#### 5.1.3 get_frequent_itemsets

In [30]:
 def get_frequent_itemsets(data, minSupport=0.2): 
  """
  Genera todos los conjuntos frecuentes, y determina el valor del support

  Parameters
  =========
    `data :` diccionario  
          ej -> {0:['A', 'B'],1:['B', 'C']} 

    `minSupport :` Soporte minimo para las relaciones (float)
          ej -> 0.5

  Returns
  =========
    `L :` Lista de elementos del tipo frozenset() (k+1 elementos)
          ej -> [frozenset({'A','B'}),frozenset({'A','C'}),frozenset({'B','C'})]
          
    `supportData :`Dicionario de conjuntos inmutables frozenset() (k+1 elemento)
          ej -> {frozenset({'A',...}): 0.5 , frozenset({'B',...}): 0.6}
    
    - Retorna todos los itemset frecuentes y sus soportes

  """
  songs, playlists = dataset_preprocessing(data)
  L1, supportData = get_first_frequent_itemset(playlists, songs, minSupport)
  L = [L1]
  #print("Total de canciones:",len(L[0]))
  k = 1
  while (len(L[k - 1]) > 0):  # Crear Ck
      #print("k==",k," cantidad:",len(L[k-1]))
      Ck = get_combination_apriori(L[k - 1], k)
      #print("Ck Generado:",len(Ck))
      #print("Ck Generado:",len(Ck[0]))
      Lk, supK = get_k_frequent_itemset(playlists, Ck, minSupport)
      supportData.update(supK)
      L.append(Lk)
      k += 1
  return L, supportData

### **5.2. Get Combination Apriori**

In [15]:
# Generar posibles conjuntos (k-ItemSet) a partir de mezclar los k-1 ItemSetFrequent
# Los ItemSet deben estar ordenados lexicográficamente
# Solo mezclamos los pares de itemsets que difieren en su último ítem.
# Por principio de poda APRIORI
def get_combination_apriori(ItemSetFrequent, k):
  """
  Genera conjuntos (k-ItemSet) a partir de mezclar los k-1 ItemSetFrequent

  Parameters
  =========
    `ItemSetFrequent :` Lista de elementos del tipo frozenset() (k=1 elementos)
          ej -> [frozenset({'A'}),frozenset({'B'}),frozenset({'C'})]

    `k :` Cantidad de elementos en cada itemset de ItemSetFrequent (int)
          ej -> 1

  Returns
  =========
    `ItemSet_k :` Lista de elementos del tipo frozenset() (k+1 elementos)
          ej -> [frozenset({'A','B'}),frozenset({'A','C'}),frozenset({'B','C'})]
          
  """
  
  ItemSet_k=[]
  tam=len(ItemSetFrequent)
  for i in range(tam):
    for j in range(i+1,tam):
      Arr1=list(ItemSetFrequent[i])[:k-1]
      Arr2=list(ItemSetFrequent[j])[:k-1]
      #Solo combinamos los itemsets que difieren en su último ítem.
      if(Arr1==Arr2):
        ItemSet_k.append(ItemSetFrequent[i]|ItemSetFrequent[j])
  return ItemSet_k


### **5.3. generate_association_rules**

#### 5.3.1. generar_reglas

In [16]:
def generar_reglas(conjunto):
  """
  Genera todas las reglas posibles a partir de un determinado conjunto.
  Combinaciones posibles

  ej. -> generar_reglas(frozenset({1,2}))
       [[frozenset({1}), frozenset({2})], [frozenset({2}), frozenset({1})]]
       
  Parameters
  =========
    `conjunto :` Conjunto (frozenset) 
          ej -> frozenset({1,2})

  Returns
  =========
    `reglas :` Lista de listas de 2 elementos, X -> Y
          ej -> [[frozenset({1}), frozenset({2})], [frozenset({2}), frozenset({1})]]

  """
  s = conjunto
  combinaciones= list(map(frozenset,chain.from_iterable(combinations(s, r) for r in range(1,len(s)))))
  reglas=[]
  for i in combinaciones:
    r=[i,conjunto-i]
    reglas.append(r)
  return reglas

In [17]:
#Ejemplo de generacion de reglas a partir de un conjunto
generar_reglas(frozenset({1,2,3}))

[[frozenset({1}), frozenset({2, 3})],
 [frozenset({2}), frozenset({1, 3})],
 [frozenset({3}), frozenset({1, 2})],
 [frozenset({1, 2}), frozenset({3})],
 [frozenset({1, 3}), frozenset({2})],
 [frozenset({2, 3}), frozenset({1})]]

####5.3.2. poda_confidence_lift

In [18]:
def poda_confidence_lift(reglas,soportes,conf_min,min_lift):
  """
  Filtra todas las reglas que su confianza >= confianza minima
  y lift >= lift minimo
       
  Parameters
  =========
    `reglas :` Lista de listas de 2 conjuntos inmutables , X -> Y
      ej ->[[(frozenset{1}), frozenset({2})], [frozenset({2}), frozenset({3})]]
    
    `soportes :`Dicionario de conjuntos inmutables frozenset()
          ej -> {frozenset({'A',...}): 0.5 , frozenset({'B',...}): 0.6}

    `conf_min :` Confianza minima para las asociaciones (float)
          ej -> 0.5

    `min_lift :` soporte minimo para las asociaciones (float)
          ej -> 0.5

  Returns
  =========
    `resultado :` Lista de listas 4 elementos: 2 conjuntos inmutables, confianza
                  y lift
      ej ->[[(frozenset{1}), frozenset({2}), 0.568,8.44],....]

  """
  resultado=[]
  for i in reglas:
    X_u_Y= soportes[i[0] | i[1]]
    if(len(i[0])==1):
      X =soportes[list(i[0])[0]]
      Y =soportes[list(i[1])[0]]
    else:
      X =soportes[i[0]]
      Y =soportes[list(i[1])[0]]
    confianza=X_u_Y/X
    if(confianza>=conf_min):
      lift=X_u_Y/(X*Y)
      if (lift>=min_lift):
        #print(lift)
        r=[i[0],i[1],X_u_Y,confianza,lift]
        resultado.append(r)
        #print(i[0],"=>",i[1])
        #print("confidence = ", confianza)
        #print("________________________________")
  return(resultado)


#### 5.3.3. verificar_confianza

In [19]:
def verificar_confianza(itemSetFrecuente,soportes,minconf,min_lift):
  """
  Filtra un itemset frecuente verifica que su confianza >= confianza minima
  si cumple recien se pueden generar todas la posibles reflas de asociacion de 
  ese itemset
       
  Parameters
  =========
    `itemSetFrecuente :` Lista de conjuntos del tipo frozenset()
          ej -> [frozenset({'A','B'}),frozenset({'A','C'}),frozenset({'B','C'})]
    
    `soportes :`Dicionario de conjuntos inmutables frozenset()
          ej -> {frozenset({'A',...}): 0.5 , frozenset({'B',...}): 0.6}

    `conf_min :` Confianza minima para las asociaciones (float)
          ej -> 0.5

    `min_lift :` soporte minimo para las asociaciones (float)
          ej -> 0.5

  Returns
  =========
    `filtro_reglas :`  Lista de conjuntos del tipo frozenset()
          ej -> [frozenset({'A','B'}),frozenset({'A','C'}),frozenset({'B','C'})]

  """
  reglas=[]
  if(len(itemSetFrecuente[0])>1):
    for i in itemSetFrecuente:
      *_,Y=i
      X=i-{Y}
      soporte_X_u_Y=soportes[i]
      if(len(itemSetFrecuente[0])==2):
        soporte_X=soportes[list(X)[0]]
      else:
        soporte_X=soportes[X]
      confianza=soporte_X_u_Y/soporte_X
      if(confianza>=minconf):
        reglas+=generar_reglas(i)
    filtro_reglas=poda_confidence_lift(reglas,soportes,minconf,min_lift)
  else:
    print("No existen reglas")
  return filtro_reglas
    

#### 5.3.4. generate_association_rules

In [20]:
def generate_association_rules(set_frecuentes,supportData,confi_min,lon_min,min_lift):
  """
  Genera Todas las reglas de asociacion que cumplan confianza  >= conf. min
  y lift >= lift minimo

  Parameters
  =========
    `set_frecuentes :` Lista de conjuntos del tipo frozenset()
          ej -> [frozenset({'A','B'}),frozenset({'A','C'}),frozenset({'B','C'})]
    
    `supportData :`Dicionario de conjuntos inmutables frozenset()
          ej -> {frozenset({'A',...}): 0.5 , frozenset({'B',...}): 0.6}

    `conf_min :` Confianza minima para las asociaciones (float)
          ej -> 0.5

    `min_lift :` soporte minimo para las asociaciones (float)
          ej -> 0.5
    
    `lon_min :` Número mínimo de elementos a considerar en las reglas (int)
          ej -> 2

  Returns
  =========
    `filtro_reglas :`Lista de listas 4 elementos: 2 conjuntos inmutables, confianza
                  y lift
          ej ->[[(frozenset{1}), frozenset({2}), 0.568,8.44],....]

  """
  k=len(set_frecuentes)-1
  TotalReglas=[]
  for i in range(lon_min-1,k):
    TotalReglas+=verificar_confianza(set_frecuentes[i],supportData,confi_min,min_lift)
  return TotalReglas

### **5.4. apriori**

In [21]:
def apriori(data,min_support=0.5, min_confidence=0.5, min_lift=1, min_length=2):
  """
  Genera Todas las reglas de asociacion

  Parameters
  =========
    `data :` diccionario  
          ej -> {0:['A', 'B'],1:['B', 'C']} 
    
    `min_support :` soporte minimo para las asociaciones (float)
          ej -> 0.5

    `conf_min :` Confianza minima para las asociaciones (float)
          ej -> 0.5

    `min_lift :` soporte minimo para las asociaciones (float)
          ej -> 1.5
    
    `lon_min :` Número mínimo de elementos a considerar en las reglas (int)
          ej -> 2

  Returns
  =========
    `Reglas :`Lista de listas 4 elementos: 2 conjuntos inmutables, confianza
                  y lift
          ej ->[[(frozenset{1}), frozenset({2}), 0.568,8.44],....]

  """
  L, supportData= get_frequent_itemsets(data,min_support)
  Reglas=generate_association_rules(L,supportData,min_confidence,min_length,min_lift)
  return Reglas



### **5.5. Imprimir reglas**

In [22]:
def print_reglas(reglas):
  """
  Imprime las reglas de asociacion, support, confidence y lift

  ej. :       

    REGLA N° 1
    Regla      : {1} => {2, 3}
    Soporte    : 100
    Confianza  : 200
    Lift:      : 5

    --------
  
  """
  cont=1
  for i in reglas:
    print('REGLA N°',cont)
    print('Regla      :',set(i[0]),"=>",set(i[1]))
    print('Soporte    :',i[2]*100)
    print('Confianza  :',i[3]*100)
    print('Lift:      :',i[4])
    print("____________________________________________")
    cont+=1

In [44]:
def print_reglas_detalle(regla,Total_playlists,n):
  """
    Imprime la regla de asociacion n con comentarios

    ej. : 
    ______________________________________________________________
    REGLA N° 1
    Regla    : {'X (feat. Future)'} => {'No Heart'}
    Veces que aparece la combinación:  103.0
    Soporte (% apariciones):  1.03
    Confianza (de 100 veces que se escucha A, la cancion B se escucha):  50.49019607843137
    Sustento (mayor a 1 -> no es casualidad):  0.5049019607843137
    ______________________________________________________________
  """
  print("______________________________________________________________")
  print('REGLA N°',n)
  print("Regla    :",set(regla[0]),"=>",set(regla[1]))
  print("Veces que aparece la combinación: " ,regla[2]*Total_playlists)
  print("Soporte (% apariciones): ",regla[2]*100)
  print("Confianza (de 100 veces que se escucha A, la cancion B se escucha): ",regla[3]*100)
  print("Sustento (mayor a 1 -> no es casualidad): ",regla[4])
  print("______________________________________________________________")

## 6.-Ejecucion del algoritmo

In [27]:
#Importamos el archivo .npy en la variable data de tipo numpy.ndarray 0-dimensional
data = np.load('DB/spotify.npy',allow_pickle=True)
#convertimos a una lista -> obtenemos un diccionario de 10000 playlists
data=data.tolist()

In [28]:
# DEFINIMOS LOS UMBRALES
lon_min = 2 # Número mínimo de canciones a considerar en las reglas
veces_minimas_aparece_en_lista=100 #Support count
soporte_min = veces_minimas_aparece_en_lista / len(data) #Support
confianza_min =0.5
sustento_min =1.5

In [31]:
# hallamos las reglas de asociacion
inicio=time.time()
association_rules=apriori(data,soporte_min,confianza_min,sustento_min,lon_min)
fin=time.time()
tiempo = fin-inicio

#### 6.1. Detalles de la ejecucion

In [34]:
print('---------------------------------------------')
print('                     DETALLES                ')
print('---------------------------------------------')
songs, playlist = dataset_preprocessing(data)
print('| Total de canciones distintas       :', len(songs))
print('| Total de Playlists(transaccciones) :', len(playlist))
print('| Total de reglas generadas          :',len(association_rules))
print('| Soporte minimo     :',soporte_min)
print('| Confianza minimo   :',confianza_min)
print('| Sutento minimo     :',sustento_min)
print('| Tiempo de ejecucion ---->',round(tiempo,4),'seg.')
print('---------------------------------------------')

---------------------------------------------
                     DETALLES                
---------------------------------------------
| Total de canciones distintas       : 132920
| Total de Playlists(transaccciones) : 10000
| Total de reglas generadas          : 83
| Soporte minimo     : 0.01
| Confianza minimo   : 0.5
| Sutento minimo     : 1.5
| Tiempo de ejecucion ----> 249.7895 seg.
---------------------------------------------


#### **6.2. Reglas de asociacion**

In [35]:
# IMPRIMIMOS TODAS LAS REGLAS DE ASOCIACION
print_reglas(association_rules)

REGLA N° 1
Regla      : {'Cold Water (feat. Justin Bieber & MØ)'} => {'Closer'}
Soporte    : 1.4500000000000002
Confianza  : 61.965811965811966
Lift:      : 8.570651724178695
____________________________________________
REGLA N° 2
Regla      : {'Chill Bill'} => {'Caroline'}
Soporte    : 1.15
Confianza  : 50.43859649122807
Lift:      : 12.932973459289247
____________________________________________
REGLA N° 3
Regla      : {'Swang'} => {'Mask Off'}
Soporte    : 1.1199999999999999
Confianza  : 51.141552511415526
Lift:      : 16.184035604878332
____________________________________________
REGLA N° 4
Regla      : {'Swang'} => {'Congratulations'}
Soporte    : 1.1199999999999999
Confianza  : 51.141552511415526
Lift:      : 12.690211541294174
____________________________________________
REGLA N° 5
Regla      : {'Tunnel Vision'} => {'Mask Off'}
Soporte    : 1.27
Confianza  : 51.417004048582996
Lift:      : 16.271203812842717
____________________________________________
REGLA N° 6
Regla      : {

#### **6.3. Top 10 reglas por lift**
Se filtro las reglas por los conjuntos que tengan un mayor lift por que un valor de lift > 1 indica que ese conjunto aparece una cantidad de veces superior a lo esperado bajo condiciones de independencia (por lo que se puede intuir que existe una relación que hace que los productos se encuentren en el conjunto más veces de lo normal).

In [36]:
# OBTENEMOS LAS 10 MEJORES REGLAS POR LIFT
rules_order_lift=sorted(association_rules,key= lambda i : i[4],reverse=True )
top_10=rules_order_lift[:10]
print_reglas(top_10)

REGLA N° 1
Regla      : {'No Heart'} => {'X (feat. Future)'}
Soporte    : 1.03
Confianza  : 69.5945945945946
Lift:      : 34.11499735029147
____________________________________________
REGLA N° 2
Regla      : {'X (feat. Future)'} => {'No Heart'}
Soporte    : 1.03
Confianza  : 50.49019607843137
Lift:      : 34.11499735029147
____________________________________________
REGLA N° 3
Regla      : {'Chicken Fried'} => {'Knee Deep (feat. Jimmy Buffett)'}
Soporte    : 1.0699999999999998
Confianza  : 50.0
Lift:      : 31.84713375796179
____________________________________________
REGLA N° 4
Regla      : {'Knee Deep (feat. Jimmy Buffett)'} => {'Chicken Fried'}
Soporte    : 1.0699999999999998
Confianza  : 68.15286624203823
Lift:      : 31.84713375796179
____________________________________________
REGLA N° 5
Regla      : {'Money Longer'} => {'You Was Right'}
Soporte    : 1.17
Confianza  : 60.30927835051546
Lift:      : 31.087256881709003
____________________________________________
REGLA N° 6
Reg

#### **6.4. Explicacion**



---


**Regla 1 : 'X (feat. Future)' => 'No Heart'**

Ambas canciones tienen : 

*   Mismo **artista** : 21 Savage, Metro Boomin
*   Mismo **álbum** : Savage Mode
*   Mismo **Género**: Hip-hop/rap

In [48]:
regla_1=top_10[1]
Total_playlists=len(playlist)
print_reglas_detalle(regla_1,Total_playlists,1)

______________________________________________________________
REGLA N° 1
Regla    : {'X (feat. Future)'} => {'No Heart'}
Veces que aparece la combinación:  103.0
Soporte (% apariciones):  1.03
Confianza (de 100 veces que se escucha A, la cancion B se escucha):  50.49019607843137
Sustento (mayor a 1 -> no es casualidad):  34.11499735029147
______________________________________________________________




---


**Regla 2 :  'Chicken Fried' => 'Knee Deep (feat. Jimmy Buffett)'**

Ambas canciones tienen : 

*   Mismo **artista** : Zac Brown Band
*   Mismo **Género**: Country

In [47]:
regla_1=top_10[2]
Total_playlists=len(playlist)
print_reglas_detalle(regla_1,Total_playlists,2)

______________________________________________________________
REGLA N° 2
Regla    : {'Chicken Fried'} => {'Knee Deep (feat. Jimmy Buffett)'}
Veces que aparece la combinación:  107.0
Soporte (% apariciones):  1.0699999999999998
Confianza (de 100 veces que se escucha A, la cancion B se escucha):  50.0
Sustento (mayor a 1 -> no es casualidad):  31.84713375796179
______________________________________________________________




---

**Regla 3 :  'You Was Right' => 'Money Longer'**

Ambas canciones tienen : 

*   Mismo **artista** : Lil Uzi Vert
*   Mismo **álbum** : Lil Uzi Vert vs. the World
*   Mismo **Género**: Hip-hop/rap
*   Fecha de **lanzamiento**: 2016

In [46]:
regla_1=top_10[5]
Total_playlists=len(playlist)
print_reglas_detalle(regla_1,Total_playlists,3)

______________________________________________________________
REGLA N° 3
Regla    : {'You Was Right'} => {'Money Longer'}
Veces que aparece la combinación:  117.0
Soporte (% apariciones):  1.17
Confianza (de 100 veces que se escucha A, la cancion B se escucha):  60.30927835051546
Sustento (mayor a 1 -> no es casualidad):  31.087256881709003
______________________________________________________________




---
**Regla 4 :  'Butterfly Effect' => 'Bank Account'**

Las canciones son de distintos artistas

Ambas canciones tienen : 

*   Mismo **Género**: Hip-hop
*   Fecha de **lanzamiento**: 2017



In [45]:
regla_1=top_10[6]
Total_playlists=len(playlist)
print_reglas_detalle(regla_1,Total_playlists,4)

______________________________________________________________
REGLA N° 4
Regla    : {'Butterfly Effect'} => {'Bank Account'}
Veces que aparece la combinación:  111.0
Soporte (% apariciones):  1.11
Confianza (de 100 veces que se escucha A, la cancion B se escucha):  57.21649484536082
Sustento (mayor a 1 -> no es casualidad):  24.876736889287315
______________________________________________________________
